## IMPORT LIBRARIES

In [1]:
import numpy as np
import tensorflow as tf
import cv2
from scipy.spatial import distance as dist
from PIL import Image
from object_detection.utils import label_map_util
import imageio


from centroid_track import CentroidTracker
from anchor import DirectionAnchor

## LOAD OUR TRAINED MODEL USING TENSORFLOW 

In [4]:
# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile('frozen_inference_graph.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
        
        

In [6]:
# Loading label map
# Label maps map indices to category names
label_map = label_map_util.load_labelmap('a.pbtxt')
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=6, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



## TAKING VIDEO INPUT USING OPENCV

In [5]:
#video File input
cap=cv2.VideoCapture('tests/video.mov')

In [9]:
#for storing tracked centroids and direction achors
ct = CentroidTracker()
DirectionObjects = {}

## INFERENCE LOOP

In [11]:
# Detection
with detection_graph.as_default():
    with imageio.get_writer('out.gif', mode='I',fps=65) as gifwriter:
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            while True:
            
                # Read frame from video file
                
                ret, image_np = cap.read()
                width  = cap.get(3) # float
                height = cap.get(4) # float
                
                # check if the frame is None,i.e. end of file if so, break out of the loop
                if image_np is None:
                    break

                
                #to store detected bounding boxes
                rects = []

                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                
                # Extract image tensor
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Extract detection boxes
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Extract detection scores
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                # Extract detection classes
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                # Extract number of detectionsd
                num_detections = detection_graph.get_tensor_by_name(
                    'num_detections:0')


                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})


                #loop over all detections and pass the valid ones to our tracker
                
                for i in range(0, boxes.shape[1]):
                    
                    if scores[0][i]>0.2:#conficence threshold

                        if boxes[0][i][0]!=0:#so that no empty detections get added

                            ymin = int((boxes[0][i][0]*height))
                            xmin = int((boxes[0][i][1]*width))
                            ymax = int((boxes[0][i][2]*height))
                            xmax = int((boxes[0][i][3]*width))



                            #visualize the detected bounding box
                            cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax),(0, 255, 0), 8)            
                            rects.append([xmax,ymax,xmin,ymin])

                #update tracking using this frames detections
                objects = ct.update(rects)



                # loop over the tracked objects
                for (objectID, centroid) in objects.items():

                    #VISUALISE the tracked centroid and tracking ID
                    cv2.circle(image_np, (centroid[0], centroid[1]), 2, (255, 255, 255), -1)
                    text = "ID {}".format(objectID)
                    cv2.putText(image_np, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX,3,(0, 255, 0),5)


                    #try to get direction achor from our list 
                    to = DirectionObjects.get(objectID, None)

                    # if there is no existing trackable object, create one


                    if to is None:

                        to = DirectionAnchor(objectID, centroid)

                    #measure distance between current centroid and direction achor centroid
                    d =dist.euclidean(tuple(centroid), tuple(to.centroids))
                    if d > 250 :
                        to.centroids = centroid#if more than threshold update anchor
                        continue
                    else:
                    #draw and arrow between anchor and current position
                        cv2.arrowedLine(image_np, tuple(to.centroids), tuple(centroid),(0, 255, 0), 10,tipLength = 0.6)
                    #store Direction anchor
                    DirectionObjects[objectID] = to

                #resize for output
                image_np=cv2.resize(image_np, (0, 0), fx = 0.2, fy = 0.2)
                gifwriter.append_data(image_np)#write to gif File